In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt
                           aes xsave avx f16c rdrand hypervisor 

**----------Load The Data---------**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math
import random
import numpy as np
from numpy import ndarray

In [4]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 9 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [5]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/CO2_QM_ds_mod_augmented.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [6]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, SQ)
]

In [26]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


------------

**--------QuickSelect Algorithm----------------**

In [8]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

---------------

**----------VP Tree---------------------**

In [9]:
import numpy as np
from collections import deque

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        """
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                pts = node._np_points if node._np_points is not None else np.array(node.points)
                dists = np.linalg.norm(pts - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = pts[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array or convert
                    tpts = node._np_threshold_points if node._np_threshold_points is not None else np.array(node.threshold_points)
                    dists = np.linalg.norm(tpts - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = tpts[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """
        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))

**------VP Tree Node Size Optimization For Rebuild------**

**------------Run 1---------------------**

In [10]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_range_start = 100
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 100

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 24.18it/s]


-------------------------------------------------------------
Build Time: 23.4373 sec, Search Time: 1.1899 sec
Leaf Size: 100, Total Time: 24.6272 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 35.11it/s]


-------------------------------------------------------------
Build Time: 21.0248 sec, Search Time: 0.8800 sec
Leaf Size: 200, Total Time: 21.9048 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 12.9251 sec, Search Time: 0.6275 sec
Leaf Size: 9,800, Total Time: 13.5526 sec

99: === Testing with Leaf Size: 9,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 50.30it/s]


-------------------------------------------------------------
Build Time: 12.9847 sec, Search Time: 0.6482 sec
Leaf Size: 9,900, Total Time: 13.6330 sec

100: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 52.23it/s]

-------------------------------------------------------------
Build Time: 13.0512 sec, Search Time: 0.6317 sec
Leaf Size: 10,000, Total Time: 13.6829 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 23.4373 sec, Search Time: 1.1899 sec, Total Time: 24.6272 sec
Leaf Size: 200, Build Time: 21.0248 sec, Search Time: 0.8800 sec, Total Time: 21.9048 sec
Leaf Size: 300, Build Time: 19.3248 sec, Search Time: 0.6804 sec, Total Time: 20.0051 sec
Leaf Size: 400, Build Time: 19.4408 sec, Search Time: 0.6920 sec, Total Time: 20.1328 sec
Leaf Size: 500, Build Time: 17.9981 sec, Search Time: 0.5812 sec, Total Time: 18.5793 sec
Leaf Size: 600, Build Time: 17.9194 sec, Search Time: 0.5799 sec, Total Time: 18.4993 sec
Leaf Size: 700, Build Time: 17.7700 sec, Search Time: 0.5977 sec, Total Time: 18.3677 sec
Leaf Size: 800, Build Time: 17.7467 sec, Search Time: 0.5826 sec, Total Time: 18.3293 sec
Leaf Size: 900, Build Time: 17.8073 sec, Search Time: 0.5998 sec, Total Time: 18.40

**---------------------Run 2---------------**

In [12]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_range_start = 100
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 100

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 20.44it/s]


-------------------------------------------------------------
Build Time: 22.8879 sec, Search Time: 1.3028 sec
Leaf Size: 100, Total Time: 24.1907 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 29.89it/s]


-------------------------------------------------------------
Build Time: 21.2301 sec, Search Time: 0.9489 sec
Leaf Size: 200, Total Time: 22.1790 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 13.0601 sec, Search Time: 0.6842 sec
Leaf Size: 9,800, Total Time: 13.7443 sec

99: === Testing with Leaf Size: 9,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 45.78it/s]


-------------------------------------------------------------
Build Time: 13.1550 sec, Search Time: 0.6843 sec
Leaf Size: 9,900, Total Time: 13.8393 sec

100: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 45.57it/s]

-------------------------------------------------------------
Build Time: 12.9762 sec, Search Time: 0.6872 sec
Leaf Size: 10,000, Total Time: 13.6634 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 22.8879 sec, Search Time: 1.3028 sec, Total Time: 24.1907 sec
Leaf Size: 200, Build Time: 21.2301 sec, Search Time: 0.9489 sec, Total Time: 22.1790 sec
Leaf Size: 300, Build Time: 19.8919 sec, Search Time: 0.7356 sec, Total Time: 20.6275 sec
Leaf Size: 400, Build Time: 19.7778 sec, Search Time: 0.7471 sec, Total Time: 20.5249 sec
Leaf Size: 500, Build Time: 18.3708 sec, Search Time: 0.6353 sec, Total Time: 19.0061 sec
Leaf Size: 600, Build Time: 18.2976 sec, Search Time: 0.6357 sec, Total Time: 18.9333 sec
Leaf Size: 700, Build Time: 18.4197 sec, Search Time: 0.6614 sec, Total Time: 19.0811 sec
Leaf Size: 800, Build Time: 18.5507 sec, Search Time: 0.6489 sec, Total Time: 19.1996 sec
Leaf Size: 900, Build Time: 18.7034 sec, Search Time: 0.6525 sec, Total Time: 19.35

**-----Opt Node Size-----**

In [13]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array(
    [  100,  200,  300,  400,  500,  600,  700,  800,  900, 1000, 1100, 1200,
  1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400,
  2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600,
  3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800,
  4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000,
  6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200,
  7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 8100, 8200, 8300, 8400,
  8500, 8600, 8700, 8800, 8900, 9000, 9100, 9200, 9300, 9400, 9500, 9600,
  9700, 9800, 9900,10000]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
    [24.6272,21.9048,20.0051,20.1328,18.5793,18.4993,18.3677,18.3293,18.407 ,
 17.1551,17.1992,17.1127,17.0445,16.9341,17.0025,16.9379,16.9723,17.108 ,
 17.1239,15.9058,15.9355,15.9782,15.9752,16.0352,16.0717,15.897 ,15.9452,
 15.9795,15.8788,15.8769,15.9052,16.009 ,16.0992,15.9809,15.9535,15.9611,
 16.0336,16.0291,15.7892,14.7913,14.7021,14.6772,14.6915,14.7423,14.8116,
 14.8199,14.6831,14.8429,14.8863,14.9213,14.7612,14.692 ,14.7138,14.6948,
 14.6568,14.6907,14.9371,14.9001,14.8145,14.7292,14.8593,14.8112,14.6551,
 14.8026,14.8799,14.7335,14.5653,14.6257,14.8774,14.8679,14.7802,15.0315,
 14.9845,14.8785,15.0203,15.0188,15.3585,15.4865,13.7492,13.7489,13.9346,
 13.9802,13.8598,13.7769,13.8146,13.6751,13.5919,13.5399,13.5626,13.5407,
 13.5025,13.6485,13.5271,13.5104,13.4268,13.5045,13.5129,13.5526,13.633 ,
 13.6829]
    )

total_times_2 = np.array(
    [24.1907,22.179 ,20.6275,20.5249,19.0061,18.9333,19.0811,19.1996,19.3559,
 17.8218,17.7316,17.704 ,17.5155,17.7903,17.5753,17.6427,17.5868,17.5552,
 17.6458,16.3089,16.3122,16.4129,16.5009,16.3107,16.2927,16.3368,16.4023,
 16.4682,16.3388,16.3622,16.3231,16.3217,16.3067,16.3452,16.2776,16.1572,
 16.4323,16.2742,16.3485,14.8448,15.0673,15.1342,14.9984,14.9357,15.164 ,
 15.0826,14.9097,14.8185,15.1363,15.0492,15.0161,15.145 ,15.2751,15.2285,
 15.0201,14.8283,15.0858,15.0038,15.0477,15.1486,15.2457,15.1692,15.1821,
 14.866 ,14.9005,15.0358,15.0398,15.0716,15.1234,14.9459,15.0493,15.0349,
 15.1752,15.2024,14.9422,15.0269,15.1098,15.0016,13.7023,13.7549,13.8199,
 13.7336,13.7634,13.7678,13.7541,13.8126,13.7123,13.7489,13.6965,13.6787,
 13.6392,13.6373,13.5901,13.804 ,13.7552,13.8309,13.7779,13.7443,13.8393,
 13.6634]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 13.5586
Minimum found at leaf size: 9300


**-----------Epsilon Optimization For Rebuild---------**

**---------------Run 1---------------------**

In [14]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size = 9300
# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 25  # Ensure inclusive range
epsilon_size_step = 0.5

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 45.78it/s]


-------------------------------------------------------------
Build Time: 13.3039 sec, Search Time: 0.6854 sec
Epsilon: 0.0, Total Time: 13.9893 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 831.05it/s]


-------------------------------------------------------------
Build Time: 5.0531 sec, Search Time: 0.0385 sec
Epsilon: 0.5, Total Time: 5.0916 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 2.0085 sec, Search Time: 0.0026 sec
Epsilon: 24.0, Total Time: 2.0110 sec

50: === Testing with Epsilon: 24.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 24.5): 100%|██████████| 20/20 [00:00<00:00, 46603.38it/s]


-------------------------------------------------------------
Build Time: 2.0103 sec, Search Time: 0.0027 sec
Epsilon: 24.5, Total Time: 2.0130 sec

51: === Testing with Epsilon: 25.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 44667.77it/s]

-------------------------------------------------------------
Build Time: 2.0004 sec, Search Time: 0.0022 sec
Epsilon: 25.0, Total Time: 2.0026 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 13.3039 sec, Search Time: 0.6854 sec, Total Time: 13.9893 sec
Epsilon: 0.5, Build Time: 5.0531 sec, Search Time: 0.0385 sec, Total Time: 5.0916 sec
Epsilon: 1.0, Build Time: 4.7004 sec, Search Time: 0.0068 sec, Total Time: 4.7072 sec
Epsilon: 1.5, Build Time: 4.6397 sec, Search Time: 0.0039 sec, Total Time: 4.6435 sec
Epsilon: 2.0, Build Time: 4.5729 sec, Search Time: 0.0042 sec, Total Time: 4.5770 sec
Epsilon: 2.5, Build Time: 4.3367 sec, Search Time: 0.0030 sec, Total Time: 4.3398 sec
Epsilon: 3.0, Build Time: 3.7136 sec, Search Time: 0.0115 sec, Total Time: 3.7251 sec
Epsilon: 3.5, Build Time: 3.2372 sec, Search Time: 0.0044 sec, Total Time: 3.2417 sec
Epsilon: 4.0, Build Time: 3.0210 sec, Search Time: 0.0029 sec, Total Time: 3.0239 sec
Epsilon: 4.5, Build Time: 2.95

**-------------Run 2------------------------**

In [16]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size = 9300
# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 25  # Ensure inclusive range
epsilon_size_step = 0.5

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 54.06it/s]


-------------------------------------------------------------
Build Time: 13.1606 sec, Search Time: 0.6207 sec
Epsilon: 0.0, Total Time: 13.7813 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 951.89it/s]


-------------------------------------------------------------
Build Time: 4.9328 sec, Search Time: 0.0353 sec
Epsilon: 0.5, Total Time: 4.9681 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 1.9806 sec, Search Time: 0.0023 sec
Epsilon: 24.0, Total Time: 1.9830 sec

50: === Testing with Epsilon: 24.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 24.5): 100%|██████████| 20/20 [00:00<00:00, 28484.24it/s]


-------------------------------------------------------------
Build Time: 2.0400 sec, Search Time: 0.0032 sec
Epsilon: 24.5, Total Time: 2.0432 sec

51: === Testing with Epsilon: 25.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 29715.22it/s]

-------------------------------------------------------------
Build Time: 2.0351 sec, Search Time: 0.0028 sec
Epsilon: 25.0, Total Time: 2.0379 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 13.1606 sec, Search Time: 0.6207 sec, Total Time: 13.7813 sec
Epsilon: 0.5, Build Time: 4.9328 sec, Search Time: 0.0353 sec, Total Time: 4.9681 sec
Epsilon: 1.0, Build Time: 4.6686 sec, Search Time: 0.0064 sec, Total Time: 4.6749 sec
Epsilon: 1.5, Build Time: 4.6737 sec, Search Time: 0.0033 sec, Total Time: 4.6770 sec
Epsilon: 2.0, Build Time: 4.6422 sec, Search Time: 0.0034 sec, Total Time: 4.6456 sec
Epsilon: 2.5, Build Time: 4.3847 sec, Search Time: 0.0038 sec, Total Time: 4.3884 sec
Epsilon: 3.0, Build Time: 3.7428 sec, Search Time: 0.0121 sec, Total Time: 3.7549 sec
Epsilon: 3.5, Build Time: 3.2488 sec, Search Time: 0.0037 sec, Total Time: 3.2524 sec
Epsilon: 4.0, Build Time: 2.9806 sec, Search Time: 0.0030 sec, Total Time: 2.9836 sec
Epsilon: 4.5, Build Time: 3.01

**-----------Epsilon Optimization For Rebuild------------**

In [17]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
    [ 0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5,
  7. , 7.5, 8. , 8.5, 9. , 9.5,10. ,10.5,11. ,11.5,12. ,12.5,13. ,13.5,
 14. ,14.5,15. ,15.5,16. ,16.5,17. ,17.5,18. ,18.5,19. ,19.5,20. ,20.5,
 21. ,21.5,22. ,22.5,23. ,23.5,24. ,24.5,25. ]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
    [13.9893, 5.0916, 4.7072, 4.6435, 4.577 , 4.3398, 3.7251, 3.2417, 3.0239,
  2.961 , 2.9668, 2.9904, 2.8572, 2.3035, 2.257 , 2.2473, 2.2627, 2.1556,
  2.1539, 2.0736, 1.9413, 2.0046, 2.0389, 1.9932, 2.0276, 2.0271, 2.    ,
  2.0386, 2.0195, 1.9857, 1.9907, 1.981 , 1.974 , 1.971 , 2.018 , 1.9674,
  1.99  , 1.9962, 1.9968, 2.0099, 1.9673, 1.954 , 1.9739, 1.9792, 1.9695,
  1.9935, 2.0548, 1.9835, 2.011 , 2.013 , 2.0026]
    )

total_times_2 = np.array(
    [13.7813, 4.9681, 4.6749, 4.677 , 4.6456, 4.3884, 3.7549, 3.2524, 2.9836,
  3.0215, 2.9543, 2.9987, 2.8666, 2.3273, 2.2735, 2.2436, 2.2467, 2.1769,
  2.1374, 2.1092, 1.9647, 1.9881, 1.9832, 1.978 , 2.011 , 2.0148, 1.9948,
  2.0012, 1.9629, 1.9738, 2.0109, 2.0338, 1.9747, 1.9755, 2.0072, 1.9748,
  2.0212, 2.0667, 2.0104, 1.9692, 2.0052, 1.9783, 1.9668, 2.0035, 2.0036,
  1.9932, 1.987 , 1.994 , 1.983 , 2.0432, 2.0379]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 1.9530
Minimum found at epsilon size: 10.0


**-------------VP Node Optimization For Insert--------**

**--------------Run 1------------------**

In [21]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 3000
leaf_size_step = 100

leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]
    # --- Build the VP Tree With Insertions---
    vptree.leaf_size = leaf_size
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 14.40it/s]


-------------------------------------------------------------
Build Time: 11.8133 sec, Search Time: 1.6498 sec
Leaf Size: 100, Total Time: 13.4630 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 14.43it/s]


-------------------------------------------------------------
Build Time: 11.8077 sec, Search Time: 1.6494 sec
Leaf Size: 200, Total Time: 13.4571 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 11.8981 sec, Search Time: 1.6610 sec
Leaf Size: 2,800, Total Time: 13.5591 sec

29: === Testing with Leaf Size: 2,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 14.37it/s]


-------------------------------------------------------------
Build Time: 11.9976 sec, Search Time: 1.6543 sec
Leaf Size: 2,900, Total Time: 13.6519 sec

30: === Testing with Leaf Size: 3,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 14.41it/s]

-------------------------------------------------------------
Build Time: 12.0450 sec, Search Time: 1.6555 sec
Leaf Size: 3,000, Total Time: 13.7005 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 11.8133 sec, Search Time: 1.6498 sec, Total Time: 13.4630 sec
Leaf Size: 200, Build Time: 11.8077 sec, Search Time: 1.6494 sec, Total Time: 13.4571 sec
Leaf Size: 300, Build Time: 11.8743 sec, Search Time: 1.6530 sec, Total Time: 13.5273 sec
Leaf Size: 400, Build Time: 11.9630 sec, Search Time: 1.6442 sec, Total Time: 13.6073 sec
Leaf Size: 500, Build Time: 11.7842 sec, Search Time: 1.6511 sec, Total Time: 13.4352 sec
Leaf Size: 600, Build Time: 11.9182 sec, Search Time: 1.6775 sec, Total Time: 13.5957 sec
Leaf Size: 700, Build Time: 11.9107 sec, Search Time: 1.6507 sec, Total Time: 13.5614 sec
Leaf Size: 800, Build Time: 11.9112 sec, Search Time: 1.6519 sec, Total Time: 13.5632 sec
Leaf Size: 900, Build Time: 11.8352 sec, Search Time: 1.6406 sec, Total Time: 13.475

**-------------Run 2---------------**

In [23]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 3000
leaf_size_step = 100

leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]
    # --- Build the VP Tree With Insertions---
    vptree.leaf_size = leaf_size
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 14.39it/s]


-------------------------------------------------------------
Build Time: 11.8641 sec, Search Time: 1.6672 sec
Leaf Size: 100, Total Time: 13.5313 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 14.32it/s]


-------------------------------------------------------------
Build Time: 11.7910 sec, Search Time: 1.6654 sec
Leaf Size: 200, Total Time: 13.4563 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 11.8947 sec, Search Time: 1.6588 sec
Leaf Size: 2,800, Total Time: 13.5535 sec

29: === Testing with Leaf Size: 2,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 14.49it/s]


-------------------------------------------------------------
Build Time: 11.8677 sec, Search Time: 1.6519 sec
Leaf Size: 2,900, Total Time: 13.5197 sec

30: === Testing with Leaf Size: 3,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 14.53it/s]

-------------------------------------------------------------
Build Time: 11.8942 sec, Search Time: 1.6394 sec
Leaf Size: 3,000, Total Time: 13.5336 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 11.8641 sec, Search Time: 1.6672 sec, Total Time: 13.5313 sec
Leaf Size: 200, Build Time: 11.7910 sec, Search Time: 1.6654 sec, Total Time: 13.4563 sec
Leaf Size: 300, Build Time: 11.8134 sec, Search Time: 1.6322 sec, Total Time: 13.4457 sec
Leaf Size: 400, Build Time: 12.0713 sec, Search Time: 1.6442 sec, Total Time: 13.7156 sec
Leaf Size: 500, Build Time: 11.8582 sec, Search Time: 1.6419 sec, Total Time: 13.5002 sec
Leaf Size: 600, Build Time: 12.0485 sec, Search Time: 1.6627 sec, Total Time: 13.7112 sec
Leaf Size: 700, Build Time: 11.9395 sec, Search Time: 1.6543 sec, Total Time: 13.5938 sec
Leaf Size: 800, Build Time: 11.6290 sec, Search Time: 1.6414 sec, Total Time: 13.2704 sec
Leaf Size: 900, Build Time: 11.8058 sec, Search Time: 1.6383 sec, Total Time: 13.444

**------Optimum Node Size for Insertion----**

In [24]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array(
    [ 100, 200, 300, 400, 500, 600, 700, 800, 900,1000,1100,1200,1300,1400,
 1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500,2600,2700,2800,
 2900,3000]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
    [13.463 ,13.4571,13.5273,13.6073,13.4352,13.5957,13.5614,13.5632,13.4757,
 13.5057,13.5296,13.3888,13.5022,13.663 ,13.6771,13.622 ,13.5326,13.5751,
 13.5616,13.6004,13.4284,13.6024,13.5081,13.513 ,13.4188,13.371 ,13.6622,
 13.5591,13.6519,13.7005]
    )

total_times_2 = np.array(
    [13.5313,13.4563,13.4457,13.7156,13.5002,13.7112,13.5938,13.2704,13.4442,
 13.4047,13.4905,13.4581,13.4623,13.5321,13.5794,13.5393,13.5093,13.5344,
 13.4211,13.5496,13.4592,13.5728,13.3745,13.4252,13.4702,13.4795,13.5495,
 13.5535,13.5197,13.5336]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 13.4168
Minimum found at leaf size: 800


**------Epsilon Optimization For Insertion------------**

**----------Run 1-----------------**

In [25]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800

# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 25  # Ensure inclusive range
epsilon_size_step = 0.5

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity
# Readjust for the insertion
data_size, query_size = test_combinations[0]

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    vptree.leaf_size = leaf_size_insert
    vptree.epsilon = epsilon

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:01<00:00, 15.23it/s]


-------------------------------------------------------------
Build Time: 12.0099 sec, Search Time: 1.5757 sec
Epsilon: 0.0, Total Time: 13.5856 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:01<00:00, 14.71it/s]


-------------------------------------------------------------
Build Time: 11.9321 sec, Search Time: 1.6263 sec
Epsilon: 0.5, Total Time: 13.5584 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 11.7977 sec, Search Time: 1.6727 sec
Epsilon: 24.0, Total Time: 13.4704 sec

50: === Testing with Epsilon: 24.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 24.5): 100%|██████████| 20/20 [00:01<00:00, 14.56it/s]


-------------------------------------------------------------
Build Time: 11.7530 sec, Search Time: 1.6380 sec
Epsilon: 24.5, Total Time: 13.3910 sec

51: === Testing with Epsilon: 25.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 14.40it/s]

-------------------------------------------------------------
Build Time: 11.7424 sec, Search Time: 1.6521 sec
Epsilon: 25.0, Total Time: 13.3945 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 12.0099 sec, Search Time: 1.5757 sec, Total Time: 13.5856 sec
Epsilon: 0.5, Build Time: 11.9321 sec, Search Time: 1.6263 sec, Total Time: 13.5584 sec
Epsilon: 1.0, Build Time: 11.9835 sec, Search Time: 1.6715 sec, Total Time: 13.6549 sec
Epsilon: 1.5, Build Time: 11.8705 sec, Search Time: 1.6673 sec, Total Time: 13.5378 sec
Epsilon: 2.0, Build Time: 12.0831 sec, Search Time: 1.6593 sec, Total Time: 13.7424 sec
Epsilon: 2.5, Build Time: 12.1422 sec, Search Time: 1.6622 sec, Total Time: 13.8044 sec
Epsilon: 3.0, Build Time: 12.0534 sec, Search Time: 1.6670 sec, Total Time: 13.7204 sec
Epsilon: 3.5, Build Time: 11.9349 sec, Search Time: 1.6716 sec, Total Time: 13.6065 sec
Epsilon: 4.0, Build Time: 12.1393 sec, Search Time: 1.6639 sec, Total Time: 13.8032 sec
Epsilon: 4.5

**----------Run 2---------------**

In [27]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800

# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 25  # Ensure inclusive range
epsilon_size_step = 0.5

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity
# Readjust for the insertion
data_size, query_size = test_combinations[0]

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    vptree.leaf_size = leaf_size_insert
    vptree.epsilon = epsilon

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:01<00:00, 14.56it/s]


-------------------------------------------------------------
Build Time: 11.5371 sec, Search Time: 1.6387 sec
Epsilon: 0.0, Total Time: 13.1758 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:01<00:00, 14.33it/s]


-------------------------------------------------------------
Build Time: 11.5890 sec, Search Time: 1.6582 sec
Epsilon: 0.5, Total Time: 13.2472 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 11.6473 sec, Search Time: 1.6535 sec
Epsilon: 24.0, Total Time: 13.3008 sec

50: === Testing with Epsilon: 24.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 24.5): 100%|██████████| 20/20 [00:01<00:00, 14.25it/s]


-------------------------------------------------------------
Build Time: 11.7336 sec, Search Time: 1.6667 sec
Epsilon: 24.5, Total Time: 13.4003 sec

51: === Testing with Epsilon: 25.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 14.43it/s]

-------------------------------------------------------------
Build Time: 11.6691 sec, Search Time: 1.6504 sec
Epsilon: 25.0, Total Time: 13.3195 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 11.5371 sec, Search Time: 1.6387 sec, Total Time: 13.1758 sec
Epsilon: 0.5, Build Time: 11.5890 sec, Search Time: 1.6582 sec, Total Time: 13.2472 sec
Epsilon: 1.0, Build Time: 11.6251 sec, Search Time: 1.6623 sec, Total Time: 13.2875 sec
Epsilon: 1.5, Build Time: 11.6067 sec, Search Time: 1.6628 sec, Total Time: 13.2695 sec
Epsilon: 2.0, Build Time: 11.5062 sec, Search Time: 1.6659 sec, Total Time: 13.1721 sec
Epsilon: 2.5, Build Time: 11.8376 sec, Search Time: 1.6549 sec, Total Time: 13.4925 sec
Epsilon: 3.0, Build Time: 11.6145 sec, Search Time: 1.6499 sec, Total Time: 13.2644 sec
Epsilon: 3.5, Build Time: 11.6350 sec, Search Time: 1.6586 sec, Total Time: 13.2935 sec
Epsilon: 4.0, Build Time: 11.6962 sec, Search Time: 1.6502 sec, Total Time: 13.3464 sec
Epsilon: 4.5

**--------Epsilon Optimization For Insert---------**

In [28]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
    [ 0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5,
  7. , 7.5, 8. , 8.5, 9. , 9.5,10. ,10.5,11. ,11.5,12. ,12.5,13. ,13.5,
 14. ,14.5,15. ,15.5,16. ,16.5,17. ,17.5,18. ,18.5,19. ,19.5,20. ,20.5,
 21. ,21.5,22. ,22.5,23. ,23.5,24. ,24.5,25. ]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
    [13.5856,13.5584,13.6549,13.5378,13.7424,13.8044,13.7204,13.6065,13.8032,
 13.6976,13.7988,13.3075,13.2966,13.3295,13.3446,13.3459,13.2447,13.4128,
 13.306 ,13.3264,13.2703,13.2958,13.5289,13.2815,13.3108,13.4028,13.3532,
 13.3284,13.2301,13.2324,13.3435,13.3705,13.5095,13.3712,13.3559,13.4327,
 13.4019,13.3745,13.3573,13.338 ,13.2962,13.2968,13.4253,13.3485,13.4156,
 13.3598,13.4326,13.3925,13.4704,13.391 ,13.3945]
    )

total_times_2 = np.array(
    [13.1758,13.2472,13.2875,13.2695,13.1721,13.4925,13.2644,13.2935,13.3464,
 13.2153,13.1589,13.2537,13.4557,13.225 ,13.4485,13.3292,13.1879,13.2661,
 13.3113,13.3021,13.2764,13.4012,13.5291,13.3406,13.3021,13.5899,13.3656,
 13.3272,13.2964,13.372 ,13.3611,13.2909,13.349 ,13.4083,13.3442,13.2598,
 13.2986,13.2759,13.4477,13.3782,13.4715,13.2676,13.2822,13.2928,13.2524,
 13.4132,13.3919,13.2487,13.3008,13.4003,13.3195]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 13.2163
Minimum found at epsilon size: 8.0
